# MAIN SCRIPT

#### Paths that must be changed are highlighted using   #---------------------------------- 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import polyfit
import seaborn as sns
import math
import os
import pickle
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin/'

## Functions

In [2]:
def comb_T2954(dataset, data_combined_T2954):
    
    
    output = dataset['T3028 Batch [#]'].unique()
    
    for m in output:
        
        T2955_ind = []
        data = dataset.loc[dataset['T3028 Batch [#]'] == m]
        data = data.reset_index()
        del data['index']
        
        data['Content_T2954 [%]']=None
        data['T2954_chiral_purity [%]']=None
        data['T2954_water-content [%]']=None
        data['R493916_in_T2954 [%]']=None
        
        for i in range(data.shape[0]):
            if pd.isnull(data.loc[i]['Batch T2955 [#]'])==False:
                T2955_ind.append(i)
        
        T2955 = []
        for i in T2955_ind:
            T2955.append(data.loc[i]['Batch T2955 [#]'])
            
        indices = T2955_ind.copy()
        indices.append(data.shape[0])

        
        lim = 0
    
        for i in indices:
            if (indices[lim]!=data.shape[0]):
                lim = lim+1
                ulim = indices[lim]
        
                weight_T2954 = 0
                content_T2954 = 0
                chiral_T2954 = 0
                water_content_T2954 = 0
                R493916_T2954 = 0
        
                for j in range(i, ulim):
                    weight_T2954 = weight_T2954 + (data['Weight T2954 [kg]'][j])
                    content_T2954 = content_T2954 + (data['Weight T2954 [kg]'][j])*(data['Content T2954 [%]'][j])
                    chiral_T2954 = chiral_T2954 + (data['Weight T2954 [kg]'][j])*(data['T2954 chiral purity [%]'][j])
                    water_content_T2954 = water_content_T2954 + (data['Weight T2954 [kg]'][j])*(data['T2954 water-content [%]'][j])
                    R493916_T2954 = R493916_T2954 + (data['Weight T2954 [kg]'][j])*(data['R493916 in T2954 [%]'][j])
            
            
                content_T2954 = content_T2954/weight_T2954
                chiral_T2954 = chiral_T2954/weight_T2954
                water_content_T2954 = water_content_T2954/weight_T2954
                R493916_T2954 = R493916_T2954/weight_T2954
            
                data.loc[[i],'Content_T2954 [%]'] = content_T2954
                data.loc[[i],'T2954_chiral_purity [%]'] = chiral_T2954
                data.loc[[i],'T2954_water-content [%]'] = water_content_T2954
                data.loc[[i],'R493916_in_T2954 [%]'] = R493916_T2954
                
        data = data.dropna(axis=0, subset=['Batch T2955 [#]'] )
        data = data.drop(['Batch T2954 [#]', 'Weight T2954 [kg]', 'Content T2954 [%]', 
                          'T2954 chiral purity [%]', 'T2954 water-content [%]',
                          'R493916 in T2954 [%]'], 1)
        
        data_combined_T2954 = data_combined_T2954.append(data, ignore_index = True)
        
    return data_combined_T2954

In [3]:
def comb_2955(dataset, data_combined_T2955):
    
    output = dataset['T3028 Batch [#]'].unique()
    
    for m in output:
        
        data = dataset.loc[dataset['T3028 Batch [#]'] == m]
        data = data.reset_index()
        del data['index']
        
        data['Weight_T2955 [kg]']=None
        data['Content_T2955 [%]']=None
        data['T2955_chiral_purity [%]']=None
        data['T2955_water-content [%]']=None
        data['R493916_in_T2955 [%]']=None
        
        weight_T2955 = 0
        content_T2955 = 0
        chiral_T2955 = 0
        water_content_T2955 = 0
        R493916_T2955 = 0
        
        for j in range(data.shape[0]):
            weight_T2955 = weight_T2955 + (data['Weight T2955 [kg]'][j])
            content_T2955 = content_T2955 + (data['Weight T2955 [kg]'][j])*(data['Content_T2954 [%]'][j])
            chiral_T2955 = chiral_T2955 + (data['Weight T2955 [kg]'][j])*(data['T2954_chiral_purity [%]'][j])
            water_content_T2955 = water_content_T2955 + (data['Weight T2955 [kg]'][j])*(data['T2954_water-content [%]'][j])
            R493916_T2955 = R493916_T2955 + (data['Weight T2955 [kg]'][j])*(data['R493916_in_T2954 [%]'][j])
            
            
        content_T2955 = content_T2955/weight_T2955
        chiral_T2955 = chiral_T2955/weight_T2955
        water_content_T2955 = water_content_T2955/weight_T2955
        R493916_T2955 = R493916_T2955/weight_T2955
        
        data.loc[0,'Weight_T2955 [kg]'] = weight_T2955
        data.loc[0,'Content_T2955 [%]'] = content_T2955
        data.loc[0,'T2955_chiral_purity [%]'] = chiral_T2955
        data.loc[0,'T2955_water-content [%]'] = water_content_T2955
        data.loc[0,'R493916_in_T2955 [%]'] = R493916_T2955
        
        data = data.dropna(axis=0, subset=['No_ChangeINDEX'])
        
        data = data.drop(['Weight T2955 [kg]', 'Content_T2954 [%]', 'T2954_chiral_purity [%]',
                          'T2954_water-content [%]', 'R493916_in_T2954 [%]' ], 1)
        
        data_combined_T2955 = data_combined_T2955.append(data, ignore_index = True)
        
        
    return data_combined_T2955 

In [4]:
def pred_model(ip_data):
    #-------------------------------------------------------------------------------
    xgboost = pickle.load(open(r'M:\TechlabsWS21\FINAL model\xgboostmodel', 'rb'))
    #-------------------------------------------------------------------------------
    resultXGB = xgboost.predict(ip_data)
    resultXGB = resultXGB.item()
    
    #-------------------------------------------------------------------------------
    randomforest = pickle.load(open(r'M:\TechlabsWS21\FINAL model\randomforestmodel', 'rb'))
    #-------------------------------------------------------------------------------
    resultRF = randomforest.predict(ip_data)
    resultRF = resultRF.item()
    
    #-------------------------------------------------------------------------------
    combIP_all = pd.read_excel(r'M:\TechlabsWS21\FINAL model\combT2955FINAL.xlsx')
    #-------------------------------------------------------------------------------
    Y_nn = combIP_all['Yield T3028 [%]']
    X_nn = combIP_all.drop(['Yield T3028 [%]', 'GRT13643L', 'GRTE7765', 'weitere_Verunreinigung'], axis=1)
    Y_nn = np.ravel(Y_nn)
    X_TrainVal, X_test, Y_TrainVal, Y_test = train_test_split(X_nn, Y_nn,test_size = 0.2,random_state = 3,shuffle = True)
    X_train, X_val, Y_train, Y_val = train_test_split(X_TrainVal, Y_TrainVal,test_size = 0.2,random_state = 3,shuffle = True)
    scaler1 = MinMaxScaler().fit(X_train)
    input_data = scaler1.transform(ip_data)
    Y_train = Y_train.reshape(-1, 1)
    scaler2 = StandardScaler().fit(Y_train)


    #-------------------------------------------------------------------------------
    json_file = open(r"M:\TechlabsWS21\FINAL model\NNmodel.json", 'r')
    #-------------------------------------------------------------------------------
    nn_model_json = json_file.read()
    json_file.close()
    nn_model = model_from_json(nn_model_json)
    #-------------------------------------------------------------------------------
    nn_model.load_weights(r"M:\TechlabsWS21\FINAL model\NNmodel.h5")
    #-------------------------------------------------------------------------------

    result = nn_model.predict(input_data)
    resultNN = scaler2.inverse_transform(result)
    resultNN = resultNN.item()

    return resultXGB, resultRF, resultNN

In [5]:
def impurity_pred(yield_pred, ip_data):
    ip_data_withYield = ip_data.copy()
    ip_data_withYield.insert(0,'Yield', yield_pred)
    
    #-------------------------------------------------------------------------------
    xgboost = pickle.load(open(r'M:\TechlabsWS21\FINAL model\xgboostIMPURITY', 'rb'))
    #-------------------------------------------------------------------------------
    
    impurity = xgboost.predict(ip_data_withYield)
    
    return impurity[0,0], impurity[0,1], impurity[0,2]

## Data processing

In [31]:
#-------------------------------------------------------------------------------
chargenbaum = pd.read_excel(r'M:\TechlabsWS21\FINAL model\input_data.xlsx')
#-------------------------------------------------------------------------------

chargenbaum.rename({'T3028 Batch\n[#]':'T3028 Batch [#]',
                    'Batch T2955            [#]':'Batch T2955 [#]',
                    'Weight T2955             [kg]':'Weight T2955 [kg]',
                    'Weight T2954  in T2955 [kg]':'Weight T2954 in T2955 [kg]',
                    'Batch T2954        [#]':'Batch T2954 [#]',
                    'Weight T2954  \n[kg]':'Weight T2954 [kg]',
                    'Yield T2955 \n[%]':'Yield T2955 [%]'
                   },axis=1, inplace=True)


cols = ['T3028 Batch [#]']
chargenbaum.loc[:,cols] = chargenbaum.loc[:,cols].ffill()
data_T2954 = pd.DataFrame()
data_T2954 = comb_T2954(chargenbaum, data_T2954)
data_T2954 = data_T2954.drop(['Batch T2955 [#]','Weight T2954 in T2955 [kg]','T2955 [kg]','Yield T2955 [%]'], 1)
data_T2955 = pd.DataFrame()
data_T2955 = comb_2955(data_T2954, data_T2955)
data_T2955 = data_T2955.drop(['T3028 Batch [#]', 'No_ChangeINDEX'], 1)
data_T2955.rename({'Weight_T2955 [kg]': 'weight',
         'Content_T2955 [%]': 'content',
         'T2955_chiral_purity [%]': 'chiral_purity',
         'T2955_water-content [%]': 'water_content',
         'R493916_in_T2955 [%]': 'R493916'}, axis=1, inplace=True)

# Input to the models

In [32]:
data_T2955

,weight,content,chiral_purity,water_content,R493916
0,511.739,98.2,95.526651,0.041777,1.382232


# Outputs of prediction models

#### Yield

In [33]:
xgb, rf, nn = pred_model(data_T2955)

In [34]:
print('Yield[%] using XGBoost regression: {} \nYield[%] using Random Forest regression: {} \nYield[%] using Neural Network regression: {}'.format(xgb, rf, nn))

Yield[%] using XGBoost regression: 81.78958129882812 
Yield[%] using Random Forest regression: 83.64627734530087 
Yield[%] using Neural Network regression: 82.2645263671875


#### Impurities

In [35]:
GRT13643L, GRTE7765, weitere_Verunreinigung = impurity_pred(nn, data_T2955)

In [36]:
print('GRT13643L [%]: {} \nGRTE7765[%]: {} \nWeitere Verunreinigung[%]: {}'.format(GRT13643L, GRTE7765, weitere_Verunreinigung))

GRT13643L [%]: 0.05452575534582138 
GRTE7765[%]: 0.15020138025283813 
Weitere Verunreinigung[%]: 0.05689673870801926


# Storing Result

In [37]:
output_data = pd.DataFrame(columns=['T3028 Batch [#]', 'Yield [%]', 'GRT13643L [%]', 'GRTE7765[%]', 'Weitere Verunreinigung[%]'])
output_data.loc[0] = [chargenbaum['T3028 Batch [#]'][0], nn, GRT13643L, GRTE7765, weitere_Verunreinigung]

output_data

,T3028 Batch [#],Yield [%],GRT13643L [%],GRTE7765[%],Weitere Verunreinigung[%]
0,S0775,82.264526,0.054526,0.150201,0.056897


In [38]:
#------------------------------------------------------------------------------------------------------
#output_data.to_excel(r'M:\TechlabsWS21\FINAL model\output_data.xlsx', index = False)
#------------------------------------------------------------------------------------------------------